<a href="https://colab.research.google.com/github/Yujinsuu/nvidia_gan/blob/main/0227_korean_sentiment_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 34.9 MB/s eta 0:00:00


In [2]:
from tensorflow.keras import layers, models
import tensorflow_datasets as tfds
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
ds = tfds.load('huggingface:nsmc')
okt = Okt()

/usr/local/lib/python3.10/dist-packages/tensorflow_datasets/core/dataset_builders/huggingface_dataset_builder.py:160: FutureWarning: list_datasets is deprecated and will be removed in the next major version of datasets. Use 'huggingface_hub.list_datasets' instead.
  hf_names = hf_datasets.list_datasets()
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/nsmc/1.1.0.incompleteYBO64A/nsmc-train.tfrecord*...:   0%|          | 0/15…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/nsmc/1.1.0.incompleteYBO64A/nsmc-test.tfrecord*...:   0%|          | 0/500…

Dataset nsmc downloaded and prepared to /root/tensorflow_datasets/nsmc/1.1.0. Subsequent calls will reuse this data.


In [4]:
def clean_str(string):
    string = re.sub(r"[^가-힣A-Za-z0-9(),!?]", " ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\!", " \! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r",", ' , ',string)
    string = re.sub(r"\s{2,}","",string)
    string = re.sub(r"\'{2,}","\'",string)
    string = re.sub(r"\'", "" , string)
    string = string.strip()
    return string.lower()

In [5]:
stop_words = ['의','이','가','은','들','는','과','와','하다','한']

In [ ]:
raw_document = []
label = []
print(len(ds['test']))
num = len(ds['test'])
for idx, data in enumerate(ds['test']):

    sentence = str(data['document'].numpy().decode())
    sentence = clean_str(sentence)
    #print(sentence)
    sentence = okt.morphs(sentence , stem = True)
    stop_removed_sentence = [word for word in sentence if not word in stop_words]

    if len(stop_removed_sentence) < 3:
        continue

    raw_document.append(stop_removed_sentence)
    label.append(data['label'].numpy())

    print(f'{idx+1} / {num}')

    #print(raw_document)
    #print(label)
    #if idx == 10:
    #    break
print(len(raw_document))

In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(raw_document)

In [8]:
print(tokenizer.word_index) # 총 등장하는 단어의 수
print(tokenizer.word_counts) # 단어들이 몇 번 등장 했는지 확인

{'영화': 1, '보다': 2, '에': 3, '을': 4, '도': 5, ',': 6, '를': 7, '없다': 8, '이다': 9, '있다': 10, '좋다': 11, '\\?': 12, '너무': 13, '다': 14, '정말': 15, '\\!': 16, '되다': 17, '만': 18, '적': 19, '진짜': 20, '같다': 21, '재밌다': 22, '로': 23, '아니다': 24, '점': 25, '으로': 26, '않다': 27, '에서': 28, '만들다': 29, '평점': 30, '나오다': 31, '연기': 32, '것': 33, '내': 34, '인': 35, '최고': 36, '그': 37, '생각': 38, '안': 39, '나': 40, '게': 41, '못': 42, '왜': 43, '스토리': 44, '사람': 45, '1': 46, '드라마': 47, '감동': 48, '때': 49, '보고': 50, '더': 51, '이렇다': 52, '고': 53, '아깝다': 54, '말': 55, '감독': 56, '\\!\\!': 57, '배우': 58, '중': 59, '까지': 60, '거': 61, '아': 62, '그냥': 63, '내용': 64, '요': 65, '주다': 66, '재미': 67, '자다': 68, '가다': 69, '재미있다': 70, '네': 71, '좀': 72, '뭐': 73, '하고': 74, '수': 75, '지루하다': 76, '시간': 77, '들다': 78, '재미없다': 79, '사랑': 80, '쓰레기': 81, '모르다': 82, '싶다': 83, '볼': 84, '지': 85, '그렇다': 86, '작품': 87, '알다': 88, '10': 89, '하나': 90, '2': 91, '다시': 92, '잘': 93, '마지막': 94, '저': 95, '정도': 96, '오다': 97, '많다': 98, '처음': 99, '하': 100, '이건': 101, '장면': 102,

In [9]:
total_word_cnt = 0
min_cnt = 3
sparse_cnt = 0
unique_sparse_cnt = 0
for key, value in tokenizer.word_counts.items():
    # key 는 단어
    # value 등장 횟수
    total_word_cnt += value # 전체 등장 단어 수 체크
    if value < min_cnt:
        sparse_cnt += value # 희소 단어 총 갯수
        unique_sparse_cnt += 1

toal_cnt_unique = len(tokenizer.word_counts)
print(f'전체 단어의 수 {total_word_cnt}')
print(f'전체 단어의 수(유니크) {toal_cnt_unique}')
print(f'전체 등장 단어 중 희소 단어가 차지하는 비율 : {(sparse_cnt/total_word_cnt) * 100}')
print(f'희소 단어(유니크) / 전체 등장 단어수(유니크)  : {(unique_sparse_cnt /len(tokenizer.word_counts)) * 100}')

전체 단어의 수 580124
전체 단어의 수(유니크) 28313
전체 등장 단어 중 희소 단어가 차지하는 비율 : 3.6393943363832557
희소 단어(유니크) / 전체 등장 단어수(유니크)  : 60.428071910429836


In [10]:
tokenizer = Tokenizer(9000)
tokenizer.fit_on_texts(raw_document)

In [11]:
print(tokenizer.word_index) # 총 등장하는 단어의 수
print(tokenizer.word_counts) # 단어들이 몇 번 등장 했는지 확인

{'영화': 1, '보다': 2, '에': 3, '을': 4, '도': 5, ',': 6, '를': 7, '없다': 8, '이다': 9, '있다': 10, '좋다': 11, '\\?': 12, '너무': 13, '다': 14, '정말': 15, '\\!': 16, '되다': 17, '만': 18, '적': 19, '진짜': 20, '같다': 21, '재밌다': 22, '로': 23, '아니다': 24, '점': 25, '으로': 26, '않다': 27, '에서': 28, '만들다': 29, '평점': 30, '나오다': 31, '연기': 32, '것': 33, '내': 34, '인': 35, '최고': 36, '그': 37, '생각': 38, '안': 39, '나': 40, '게': 41, '못': 42, '왜': 43, '스토리': 44, '사람': 45, '1': 46, '드라마': 47, '감동': 48, '때': 49, '보고': 50, '더': 51, '이렇다': 52, '고': 53, '아깝다': 54, '말': 55, '감독': 56, '\\!\\!': 57, '배우': 58, '중': 59, '까지': 60, '거': 61, '아': 62, '그냥': 63, '내용': 64, '요': 65, '주다': 66, '재미': 67, '자다': 68, '가다': 69, '재미있다': 70, '네': 71, '좀': 72, '뭐': 73, '하고': 74, '수': 75, '지루하다': 76, '시간': 77, '들다': 78, '재미없다': 79, '사랑': 80, '쓰레기': 81, '모르다': 82, '싶다': 83, '볼': 84, '지': 85, '그렇다': 86, '작품': 87, '알다': 88, '10': 89, '하나': 90, '2': 91, '다시': 92, '잘': 93, '마지막': 94, '저': 95, '정도': 96, '오다': 97, '많다': 98, '처음': 99, '하': 100, '이건': 101, '장면': 102,

In [12]:
sequence = tokenizer.texts_to_sequences(raw_document)

In [18]:
drop_sequence_idx = [idx for idx , sentence in enumerate(sequence) if len(sentence) < 3 ]
obj_sequence = np.array(sequence, dtype='object')

In [19]:
np_sequence = np.delete(obj_sequence , drop_sequence_idx , axis=0)
np_label = np.delete(label , drop_sequence_idx, axis = 0)

In [21]:
len(np_sequence),len(np_label)

(45775, 45775)

In [22]:
pad_np_sequence = pad_sequences(np_sequence , maxlen = 82)

In [23]:
pad_np_sequence.shape

(45775, 82)

In [24]:
pad_np_sequence[:3]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,   73,  180,   30,  409,   27,   89,   25,
         319,  778,   51,  883,   24],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,

In [25]:
model = models.Sequential()
model.add(layers.Embedding(9000, 100))
model.add(layers.LSTM(64))
model.add(layers.Dense(1, activation='sigmoid'))

In [26]:
model.compile(optimizer='adam' , loss='binary_crossentropy', metrics=['accuracy'])
model.fit(pad_np_sequence, np_label, epochs=2 , validation_split=0.2)

Epoch 1/2
1145/1145 [==============================] - 88s 72ms/step - loss: 0.4232 - accuracy: 0.8053 - val_loss: 0.3756 - val_accuracy: 0.8322
Epoch 2/2
1145/1145 [==============================] - 81s 70ms/step - loss: 0.3161 - accuracy: 0.8650 - val_loss: 0.3772 - val_accuracy: 0.8327


In [35]:
sentence = '아 이 영화 진짜 재미없네'
sentence = clean_str(sentence)
print(sentence)
sentence = okt.morphs(sentence , stem = True)
stop_removed_sentence = [word for word in sentence if not word in stop_words]
print(stop_removed_sentence)
tokenizer = Tokenizer()
sequence = tokenizer.texts_to_sequences(stop_removed_sentence)
drop_sequence_idx = [idx for idx , sentence in enumerate(sequence) if len(sentence) < 3 ]
np_sequence = np.delete(obj_sequence , drop_sequence_idx , axis=0)
pad_np_sequence = pad_sequences(np_sequence , maxlen = 82)
print(pad_np_sequence)
pred = model.predict(pad_np_sequence)
if pred[0][0] > 0.5:
  print(f'{pred[0][0] * 100 :=0.2f}% 확률로 긍정적인 리뷰입니다. ')
else:
  print(f'{(1-pred[0][0]) * 100 :=0.2f}% 확률로 부정적인 리뷰입니다. ')

아 이 영화 진짜 재미없네
['아', '영화', '진짜', '재미없다']
[[   0    0    0 ... 3508    7 2639]
 [   0    0    0 ...   62 2490    9]
 [   0    0    0 ...   45   86   27]
 ...
 [   0    0    0 ...  303 4637   29]
 [   0    0    0 ...   28   14   71]
 [   0    0    0 ...  152   43 1173]]
1449/1449 [==============================] - 24s 16ms/step
75.64% 확률로 부정적인 리뷰입니다. 
